## **0. Подготовка окружения**




In [24]:
# from google.colab import drive
# drive.mount('/content/drive')

In [25]:
# Путь к папке с данными
DATASET_PATH = "./data"  # внутри: папки 1,2,...,8

# !pip install mediapipe opencv-python scikit-learn

In [26]:
import os
import cv2
import numpy as np
import mediapipe as mp
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
# from google.colab.patches import cv2_imshow
import random

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)


I0000 00:00:1766769714.254025 11614655 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M1 Pro


In [ ]:
import numpy as np
import cv2

def _safe_angle(a, b, c):
    ba = a - b
    bc = c - b
    denom = (np.linalg.norm(ba) * np.linalg.norm(bc)) + 1e-6
    cosv = float(np.dot(ba, bc) / denom)
    cosv = float(np.clip(cosv, -1.0, 1.0))
    return float(np.arccos(cosv))

def extract_pose_vector(frame, min_mean_vis=0.35):
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    res = pose.process(img_rgb)
    if not res.pose_landmarks:
        return None

    lms = res.pose_landmarks.landmark

    lh_i = mp_pose.PoseLandmark.LEFT_HIP.value
    rh_i = mp_pose.PoseLandmark.RIGHT_HIP.value
    ls_i = mp_pose.PoseLandmark.LEFT_SHOULDER.value
    rs_i = mp_pose.PoseLandmark.RIGHT_SHOULDER.value

    cx = 0.5 * (lms[lh_i].x + lms[rh_i].x)
    cy = 0.5 * (lms[lh_i].y + lms[rh_i].y)

    scale = np.hypot(lms[ls_i].x - lms[rs_i].x, lms[ls_i].y - lms[rs_i].y)
    if scale < 1e-6:
        scale = np.hypot(lms[ls_i].x - cx, lms[ls_i].y - cy)
    if scale < 1e-6:
        return None

    pts = np.zeros((33, 2), dtype=np.float32)
    vis = np.zeros((33,), dtype=np.float32)

    for i, lm in enumerate(lms):
        pts[i, 0] = (lm.x - cx) / scale
        pts[i, 1] = (lm.y - cy) / scale
        vis[i] = lm.visibility

    mean_vis = float(np.mean(vis))
    if mean_vis < float(min_mean_vis):
        return None

    vec = pts.reshape(-1).astype(np.float32)

    def p(idx):
        return pts[idx].astype(np.float32)

    le = mp_pose.PoseLandmark.LEFT_ELBOW.value
    re = mp_pose.PoseLandmark.RIGHT_ELBOW.value
    lw = mp_pose.PoseLandmark.LEFT_WRIST.value
    rw = mp_pose.PoseLandmark.RIGHT_WRIST.value
    lk = mp_pose.PoseLandmark.LEFT_KNEE.value
    rk = mp_pose.PoseLandmark.RIGHT_KNEE.value
    la = mp_pose.PoseLandmark.LEFT_ANKLE.value
    ra = mp_pose.PoseLandmark.RIGHT_ANKLE.value

    ang = [
        _safe_angle(p(ls_i), p(le), p(lw)),
        _safe_angle(p(rs_i), p(re), p(rw)),
        _safe_angle(p(lh_i), p(lk), p(la)),
        _safe_angle(p(rh_i), p(rk), p(ra)),
        _safe_angle(p(le), p(ls_i), p(lh_i)),
        _safe_angle(p(re), p(rs_i), p(rh_i)),
        _safe_angle(p(ls_i), p(lh_i), p(lk)),
        _safe_angle(p(rs_i), p(rh_i), p(rk)),
    ]

    out = np.concatenate([vec, np.array(ang, dtype=np.float32), np.array([mean_vis], dtype=np.float32)])
    return out


In [28]:
print("Содержимое DATASET_PATH:")
!ls "$DATASET_PATH"


W0000 00:00:1766769714.407055 11870786 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1766769714.435238 11870790 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Содержимое DATASET_PATH:
1         3         5         7         model.pkl
2         4         6         8


In [29]:
from torchvision import transforms
from PIL import Image

# Создаем трансформер
augmentations = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),      # Отражаем слева направо
    transforms.RandomRotation(degrees=15),       # Крутим на +- 15 градусов
    transforms.ColorJitter(brightness=0.2),      # Меняем яркость
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.7, 1.0)) # ГЛАВНОЕ: зум и обрезка
])

In [30]:
X = []
y = []

video_ext = (".mp4", ".avi", ".mov", ".mkv")

for class_name in sorted(os.listdir(DATASET_PATH)):
    class_path = os.path.join(DATASET_PATH, class_name)
    if not os.path.isdir(class_path):
        continue

    print(f"→ Класс {class_name}")

    for fname in sorted(os.listdir(class_path)):
        if not fname.lower().endswith(video_ext):
            continue

        video_path = os.path.join(class_path, fname)
        print(f"    Видео: {fname}")

        cap = cv2.VideoCapture(video_path)
        frame_id = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # 1. ТЕПЕРЬ БЕРЕМ КАЖДЫЙ 2-й КАДР (в 5 раз больше данных, чем было)
            if frame_id % 2 == 0:
                
                # --- БЛОК АУГМЕНТАЦИИ ---
                # Создаем копию кадра, чтобы не портить оригинал для следующих шагов
                aug_frame = frame.copy()

                # А. Рандомное отражение (слева-направо)
                # Это самое важное, чтобы модель не привыкала, что рука в одном углу
                if random.random() > 0.5:
                    aug_frame = cv2.flip(aug_frame, 1)

                # Б. Рандомный поворот (от -15 до 15 градусов)
                angle = random.uniform(-15, 15)
                h, w = aug_frame.shape[:2]
                M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
                aug_frame = cv2.warpAffine(aug_frame, M, (w, h))

                # В. Рандомное изменение яркости (чтобы не привязываться к свету)
                brightness = random.uniform(0.7, 1.3)
                aug_frame = cv2.convertScaleAbs(aug_frame, alpha=brightness, beta=0)
                
                # --- ИЗВЛЕЧЕНИЕ ВЕКТОРА ---
                # Передаем уже измененный кадр
                vec = extract_pose_vector(aug_frame)
                
                if vec is not None:
                    X.append(vec)
                    y.append(class_name)

            frame_id += 1

        cap.release()

X = np.array(X)
y = np.array(y)

print("Всего поз:", len(X))
print("Классы:", np.unique(y))


→ Класс 1
    Видео: 1_1.mov
    Видео: 1_2.mov
    Видео: 1_3.mov
    Видео: 1_4.MOV
    Видео: 1_5.mp4
→ Класс 2
    Видео: 2_1.mov
    Видео: 2_2.mov
    Видео: 2_3.mov
    Видео: 2_4.MOV
    Видео: 2_5.mp4
→ Класс 3
    Видео: 3_1.mov
    Видео: 3_2.mov
    Видео: 3_3.mov
    Видео: 3_4.MOV
    Видео: 3_5.mp4
→ Класс 4
    Видео: 4_1.mov
    Видео: 4_2.mov
    Видео: 4_3.mov
    Видео: 4_4.MOV
    Видео: 4_5.mp4
→ Класс 5
    Видео: 5_1.mov
    Видео: 5_2.mov
    Видео: 5_3.mov
    Видео: 5_4.MOV
    Видео: 5_5.mp4
→ Класс 6
    Видео: 6_1.mov
    Видео: 6_2.mov
    Видео: 6_3.mov
    Видео: 6_4.MOV
    Видео: 6_5.mp4
→ Класс 7
    Видео: 7_1.mov
    Видео: 7_2.mov
    Видео: 7_3.mov
    Видео: 7_4.MOV
    Видео: 7_5.mp4
→ Класс 8
    Видео: 8_1.mov
    Видео: 8_2.mov
    Видео: 8_3.mov
    Видео: 8_4.MOV
    Видео: 8_5.mp4
Всего поз: 5008
Классы: ['1' '2' '3' '4' '5' '6' '7' '8']


In [31]:
if len(np.unique(y)) < 2:
    raise ValueError(f"Нашли только один класс: {np.unique(y)}. Проверь структуру папок и данные.")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# уменьшаем размерность, чтобы SVM жил полегче
pca = PCA(n_components=min(40, X_train_scaled.shape[1]))
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)


In [32]:
clf = SVC(kernel="rbf", probability=True)
clf.fit(X_train_pca, y_train)

y_pred = clf.predict(X_test_pca)
print("Отчёт по качеству:\n")
print(classification_report(y_test, y_pred))
print("Матрица ошибок:")
print(confusion_matrix(y_test, y_pred))


Отчёт по качеству:

              precision    recall  f1-score   support

           1       0.64      0.79      0.71       170
           2       0.59      0.86      0.70       101
           3       0.83      0.85      0.84       152
           4       0.98      1.00      0.99        62
           5       0.90      0.75      0.82       122
           6       0.83      0.62      0.71       148
           7       0.86      0.85      0.85       169
           8       0.93      0.54      0.68        78

    accuracy                           0.78      1002
   macro avg       0.82      0.78      0.79      1002
weighted avg       0.80      0.78      0.78      1002

Матрица ошибок:
[[134  16  12   0   1   6   0   1]
 [  8  87   0   0   0   6   0   0]
 [  6   6 129   0   1   0   9   1]
 [  0   0   0  62   0   0   0   0]
 [  5   3   3   0  92   5  14   0]
 [ 40   8   0   0   7  92   0   1]
 [  3   9  11   0   1   1 144   0]
 [ 13  19   1   1   0   1   1  42]]


In [33]:
model = {
    "clf": clf,
    "scaler": scaler,
    "pca": pca,
    "classes": sorted(list(np.unique(y)))
}

model_path = os.path.join(DATASET_PATH, "model.pkl")
with open(model_path, "wb") as f:
    pickle.dump(model, f)

print("Модель сохранена в:", model_path)


Модель сохранена в: ./data/model.pkl
